In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import urllib.parse
import requests
import pickle
import time
from bs4 import BeautifulSoup

In [467]:
circuitos_2015 = pd.read_csv('cba.circuitos.2013.csv')
circuitos_2011 = pd.read_csv('cba.circuitos.2011.csv')

In [468]:
circuitos_2015 = circuitos_2015.Name[1:].tolist()
circuitos_2011 = circuitos_2011.x.tolist()

In [446]:
url_2015 = 'https://www.justiciacordoba.gob.ar/jel/ReportesEleccion20150705/Resultados/E20150705_C'

In [465]:
url_2011 = 'http://www.justiciacordoba.gob.ar/jel/ReportesEleccion20110807/Resultados/E20110807_C'

In [470]:
paginas = []
sleep = 0.1
suffix_2015 = '_CA2_0.htm'
suffix_2011 = '%20%20%20%20_CA2_0.htm'

for circuito in circuitos_2011[:3]:
    time.sleep(sleep)
    link = '{}{}{}'.format(url_2011, circuito, suffix_2011)
    print(link)
    response_iterative = requests.request("GET", link)
    paginas.append(response_iterative.text)
#    with open("cba_elecciones_gobernador.txt", "wb") as fp:
#        pickle.dump(paginas, fp)


http://www.justiciacordoba.gob.ar/jel/ReportesEleccion20110807/Resultados/E20110807_C10C%20%20%20%20_CA2_0.htm
http://www.justiciacordoba.gob.ar/jel/ReportesEleccion20110807/Resultados/E20110807_C12C%20%20%20%20_CA2_0.htm
http://www.justiciacordoba.gob.ar/jel/ReportesEleccion20110807/Resultados/E20110807_C5E%20%20%20%20_CA2_0.htm


In [449]:
data = []
data_base = pd.DataFrame()

for pagina in paginas:
    soup = BeautifulSoup(pagina, 'html.parser')
    table_body = soup.find('table')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.replace('\xa0', ' ').strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    tabla = pd.DataFrame(data)
    tabla_circuito = pd.concat((data_base, tabla), axis = 0)
    tabla_circuito.columns = ['lista', 'nombre', 'votos']
    tabla_circuito = tabla_circuito.dropna(axis = 0, how = 'all')
    totales = tabla_circuito[tabla_circuito['lista'].str.contains('Total')].shift(1, axis = 1)
    totales['lista'] = totales['lista'].fillna(value = 'None')
    tabla_circuito = pd.concat((tabla_circuito, totales), axis=0)
    tabla_circuito.sort_index().reset_index()

In [450]:
tabla_circuito = tabla_circuito.sort_index()

In [451]:
tabla_circuito['circuito'] = tabla_circuito[tabla_circuito['lista'].str.contains('Circuito')]['lista']
tabla_circuito['circuito'] = tabla_circuito['circuito'].fillna(method = 'ffill')

In [452]:
tabla_circuito = tabla_circuito.dropna(how = 'any')
tabla_circuito = tabla_circuito.drop(tabla_circuito[tabla_circuito['votos'] == 'Votos'].index)

In [453]:
tabla_circuito['votos'] = pd.to_numeric(tabla_circuito.votos.str.replace(',', ''))

In [454]:
tabla_circuito['circuito'] = tabla_circuito.circuito.str.replace('Circuito ', '')

In [455]:
circuitos_desagregado =  tabla_circuito.circuito.str.split('-', expand = True)

In [456]:
tabla_circuito = pd.concat((tabla_circuito, circuitos_desagregado), axis= 1)

In [457]:
tabla_circuito.columns = ['lista', 'nombre', 'votos', 'circuito', 'circuito_numero', 'circuito_nombre']

In [471]:
tabla_circuito = tabla_circuito.pivot_table(index = ['circuito_numero', 'circuito_nombre'], columns= 'nombre', values = 'votos').reset_index()

KeyError: 'votos'

In [460]:
tabla_circuito.to_csv('cba_gobernador_2015_circuitos.csv', index = False)